## Task6:Hit and Run Accident Investigation

In [1]:
# ======================= Task 6：肇事逃逸调查（相对路径+性能优化版） =======================
# 目标：查询 2010-08-11 当天，零件 K5-112-1122-79 对应车辆的注册地
# 优化点：在注册表中过滤时，先按车辆ID筛选，再解析日期（显著提速）

import pandas as pd
from pathlib import Path
import re, csv, warnings

# ---- 静默与显示设置（去除烦人的日期解析告警）----
warnings.filterwarnings("ignore", message="Parsing dates in %Y-%m-%d format")
pd.set_option("display.max_rows", 20)

# ---------- 自动检测项目根目录 ----------
def detect_project_root() -> Path:
    here = Path.cwd()
    for p in [here, *here.parents]:
        if (p / "Data").exists():   # 如果找到 Data 文件夹，就认为是项目根目录
            return p
    return here  # 找不到就用当前目录

PROJECT_ROOT = detect_project_root()
BASE_DIR = PROJECT_ROOT / "Data"   # 相对路径：统一指向项目下的 Data 文件夹

print("✅ Projektwurzelverzeichnis:", PROJECT_ROOT)
print("✅ Data-Verzeichnis:", BASE_DIR)

# ---------- 基本配置 ----------
TARGET_PART_RAW = "K5-112-1122-79"
QUERY_DAY = pd.to_datetime("2010-08-11")
CHUNKSIZE = 200_000  # 分块读取大小

# ---------- 列名候选（含你数据里的真实命名） ----------
VEH_ID_CANDS = [
    "idnummer","fahrzeug_id","fahrzeugid","vehicle_id","vehicleid","vin",
    "fahrzeug","fahrzeugnr","id_fahrzeug","fahrzeugnummer","fz_id","fzg_id","fzgid"
]
REG_START_CANDS = ["zulassung","zulassungsdatum","anmeldedatum","start","begin"]
REG_END_CANDS   = ["abmeldedatum","abmeldung","ende","end"]
REG_REGION_CANDS= ["gemeinden","gemeinde","zulassungsbezirk","ort","region","kreis","amt","bezirk"]

PART_CANDS = [
    "teilnummer","teilenummer","partno","part_number","teilenr",
    "komponente","bauteil","karosserieteil","bodypart","einzelteil","einzelteil_id","einzelteilnummer",
    "id_karosserie","id_k5","id_karosserie.x"   # 结合你日志中的命中列
]

# ---------- 工具函数 ----------
def canon_text(s: str) -> str:
    """标准化零件号：统一破折号、去非字母数字、转大写。"""
    if s is None: return ""
    s = str(s).strip().upper()
    s = s.replace("–","-").replace("—","-").replace("−","-").replace("‐","-")
    return re.sub(r"[^A-Z0-9]", "", s)

TARGET_PART_CANON = canon_text(TARGET_PART_RAW)

def norm_col(s: str) -> str:
    if s is None: return ""
    s = str(s).strip().lower()
    s = s.replace("ä","a").replace("ö","o").replace("ü","u").replace("ß","ss")
    return re.sub(r"[^a-z0-9]", "", s)

def first_match_col(cols, cands):
    cols = [str(c) for c in cols]
    mp = {norm_col(c): c for c in cols}
    # 完全匹配
    for cand in cands:
        k = norm_col(cand)
        if k in mp:
            return mp[k]
    # 宽松包含
    for cand in cands:
        k = norm_col(cand)
        for nk, orig in mp.items():
            if k in nk:
                return orig
    return None

def sniff_sep(filepath):
    """尝试猜 CSV 分隔符；失败返回 None。"""
    try:
        with open(filepath, "r", newline="", encoding="utf-8", errors="ignore") as f:
            sample = f.read(4096)
        dialect = csv.Sniffer().sniff(sample, delimiters=",;\t|")
        return dialect.delimiter
    except Exception:
        return None

def safe_read_csv(path, usecols=None, nrows=None, chunksize=None, dtype=str):
    """
    稳健读取 CSV：engine='python' 自动推断 → sniff → ';' → ','。
    成功返回 DataFrame 或 迭代器；失败返回 None。
    """
    try:
        return pd.read_csv(path, usecols=usecols, nrows=nrows, chunksize=chunksize,
                           dtype=dtype, low_memory=False, engine="python", sep=None)
    except Exception:
        sep = sniff_sep(path)
        for sp in [sep, ";", ",", "\t", "|"]:
            if not sp: 
                continue
            try:
                return pd.read_csv(path, usecols=usecols, nrows=nrows, chunksize=chunksize,
                                   dtype=dtype, low_memory=False, sep=sp)
            except Exception:
                continue
        return None

def preview_columns(path):
    try:
        if path.suffix.lower()==".csv":
            df = safe_read_csv(path, nrows=5)
            if df is None: return []
            if hasattr(df, "__iter__") and not isinstance(df, pd.DataFrame):
                df = next(df)
            return [str(c) for c in df.columns]
        else:
            df = pd.read_excel(path, nrows=5, engine=None)
            return [str(c) for c in df.columns]
    except Exception:
        return []

def iter_all_files(root: Path):
    """全目录递归找 CSV / XLS / XLSX。"""
    return list(root.rglob("*.csv")) + list(root.rglob("*.xls")) + list(root.rglob("*.xlsx"))

def parse_date_series(s):
    """
    智能日期解析：
    1) 先按 YYYY-MM-DD 解析；
    2) 对没解析成功的，再用 dayfirst=True 兜底（如 DD.MM.YYYY / DD/MM/YYYY）。
    """
    s1 = pd.to_datetime(s, errors="coerce", format="%Y-%m-%d")
    if isinstance(s1, pd.Series) and s1.isna().any():
        idx = s1.isna()
        s1.loc[idx] = pd.to_datetime(s[idx], errors="coerce", dayfirst=True)
    return s1

# ================== A. 全目录扫描：找到包含目标零件的文件 → 抽取车辆ID ==================
print(">>> Scanne Data-Verzeichnis zur Lokalisierung des Teils… (zeigt Trefferdateien, Spalten und Trefferanzahl)")
veh_ids = set()
hits_detail = []

all_files = iter_all_files(BASE_DIR)
# 把 Zulassungen 相关文件放到后面，避免误当零件表
all_files = sorted(all_files, key=lambda p: ("zulass" in p.as_posix().lower(), p.name.lower()))

for f in all_files:
    cols = preview_columns(f)
    if not cols: 
        continue
    part_col = first_match_col(cols, PART_CANDS)
    veh_col  = first_match_col(cols, VEH_ID_CANDS)

    try:
        if f.suffix.lower()==".csv":
            local_cnt = 0
            # 1) 先用“猜到的 part 列”直接查
            if part_col:
                it = safe_read_csv(f, usecols=[part_col]+([veh_col] if veh_col else []),
                                   chunksize=CHUNKSIZE, dtype=str)
                if it is not None:
                    for chunk in it:
                        chunk = chunk.fillna("")
                        m = chunk[part_col].astype(str).map(canon_text).eq(TARGET_PART_CANON)
                        if m.any():
                            local_cnt += int(m.sum())
                            if veh_col and veh_col in chunk.columns:
                                veh_ids.update(chunk.loc[m, veh_col].astype(str))
            # 2) 若未命中或没识别到 part 列，宽松扫描所有字符串列
            if local_cnt==0:
                head = safe_read_csv(f, nrows=1000)
                if head is not None:
                    if hasattr(head, "__iter__") and not isinstance(head, pd.DataFrame):
                        head = next(head)
                    str_cols = [c for c in head.columns if head[c].dtype==object]
                    if str_cols:
                        it = safe_read_csv(f, usecols=str_cols, chunksize=CHUNKSIZE, dtype=str)
                        if it is not None:
                            col_hits = {}
                            for chunk in it:
                                chunk = chunk.fillna("")
                                for c in str_cols:
                                    m = chunk[c].astype(str).map(canon_text).eq(TARGET_PART_CANON)
                                    if m.any():
                                        cnt = int(m.sum())
                                        col_hits[c] = col_hits.get(c,0)+cnt
                                        if not veh_col:
                                            veh_col = first_match_col(chunk.columns, VEH_ID_CANDS)
                                        if veh_col and veh_col in chunk.columns:
                                            veh_ids.update(chunk.loc[m, veh_col].astype(str))
                                        local_cnt += cnt
                            if local_cnt:
                                part_col = max(col_hits.items(), key=lambda kv: kv[1])[0]
            if local_cnt:
                hits_detail.append((f, part_col, veh_col, local_cnt))
                print(f"    [Treffer] {f.relative_to(BASE_DIR)} | Teil-Spalte={part_col} | Fahrzeug-Spalte={veh_col} | Treffer={local_cnt}")

        else:
            # Excel
            df = pd.read_excel(f, engine=None).fillna("")
            local_cnt = 0
            if part_col and part_col in df.columns:
                m = df[part_col].astype(str).map(canon_text).eq(TARGET_PART_CANON)
                local_cnt = int(m.sum())
                if local_cnt and veh_col and veh_col in df.columns:
                    veh_ids.update(df.loc[m, veh_col].astype(str))
            if local_cnt==0:
                for c in df.columns:
                    if df[c].dtype==object:
                        m = df[c].astype(str).map(canon_text).eq(TARGET_PART_CANON)
                        if m.any():
                            cnt = int(m.sum()); local_cnt += cnt; part_col = c
                            if not veh_col:
                                veh_col = first_match_col(df.columns, VEH_ID_CANDS)
                            if veh_col and veh_col in df.columns:
                                veh_ids.update(df.loc[m, veh_col].astype(str))
            if local_cnt:
                hits_detail.append((f, part_col, veh_col, local_cnt))
                print(f"    [Treffer] {f.relative_to(BASE_DIR)} | Teil-Spalte={part_col} | Fahrzeug-Spalte={veh_col} | Treffer={local_cnt}")

    except Exception as e:
        print(f"    [Scan-Fehler] {f.relative_to(BASE_DIR)} -> {e}")

veh_ids = sorted([v for v in map(str, veh_ids) if v and v.lower()!="nan"])
print(f"\n>>> Insgesamt {len(hits_detail)} Dateien getroffen; Anzahl gefundener Fahrzeug-IDs: {len(veh_ids)} (Beispiel) {veh_ids[:10]}")
if not veh_ids:
    raise SystemExit("❌ Keine Aufzeichnungen mit diesem Teil gefunden.")

# ================== B. 在注册表里筛选（先按ID过滤，再解析日期 —— 提速关键） ==================
reg_path = BASE_DIR / "Zulassungen" / "Zulassungen_alle_Fahrzeuge.csv"
if not reg_path.exists():
    raise SystemExit(f"❌ Zulassungsdatei nicht gefunden: {reg_path}")

cols_reg = preview_columns(reg_path)
col_fzg   = first_match_col(cols_reg, VEH_ID_CANDS)     # e.g., IDNummer
col_start = first_match_col(cols_reg, REG_START_CANDS)  # e.g., Zulassung
col_end   = first_match_col(cols_reg, REG_END_CANDS)    # 可能不存在
col_reg   = first_match_col(cols_reg, REG_REGION_CANDS) # e.g., Gemeinden

if not col_fzg or not col_start or not col_reg:
    raise SystemExit(f"❌ Erkennung der Schlüsselspalten fehlgeschlagen. Spaltenvorschau: {cols_reg}")

print("\n>>> Filtere gültige Zulassungen am Unfalltag (mit Fortschritt)…")
rows = []
usecols = [col_fzg, col_start, col_reg] + ([col_end] if col_end else [])
it = safe_read_csv(reg_path, usecols=usecols, chunksize=CHUNKSIZE, dtype=str)
if it is None:
    raise SystemExit("❌ Zulassungsdatei konnte nicht gelesen werden (Trennzeichen/Encoding).")

veh_ids_set = set(veh_ids)  # 仅构建一次集合，加速 isin 判断
count = 0
for chunk in it:
    count += len(chunk)
    # 降低打印频率：每处理整百万行提示一次
    if count // 1_000_000 != (count - len(chunk)) // 1_000_000:
        print(f"    [Fortschritt] {reg_path.name} {count:,} Zeilen verarbeitet…")

    # 先把可能用到的列变为 string，避免反复 astype
    for c in [col_fzg, col_start] + ([col_end] if col_end else []) + [col_reg]:
        if c in chunk.columns:
            chunk[c] = chunk[c].astype("string")

    # ① 先按车辆ID过滤（大幅减少行数）
    mask_id = chunk[col_fzg].isin(veh_ids_set)
    if not mask_id.any():
        continue

    # ② 只对命中ID的行解析日期（显著提速）
    starts = parse_date_series(chunk.loc[mask_id, col_start])
    if col_end and col_end in chunk.columns:
        ends = parse_date_series(chunk.loc[mask_id, col_end])
    else:
        ends = None

    # ③ 组条件：start ≤ 查询日 ≤ end(或无end)
    if ends is not None:
        mask_end_ok = ends.isna() | (ends >= QUERY_DAY)
        mask_day = (starts <= QUERY_DAY) & mask_end_ok
    else:
        mask_day = (starts <= QUERY_DAY)

    if not mask_day.any():
        continue

    # ④ 取最终命中行
    hit_idx = chunk.index[mask_id][mask_day]
    hit = chunk.loc[hit_idx, [col_fzg, col_reg, col_start]].copy()
    hit["Abmeldedatum"] = ends.loc[mask_day].values if ends is not None else pd.NaT
    hit["source_file"] = reg_path.relative_to(BASE_DIR).as_posix()
    rows.append(hit)

if not rows:
    raise SystemExit("❌ Keine gültigen Zulassungsdatensätze am 2010-08-11 gefunden.")

result = pd.concat(rows, ignore_index=True)

# 同一车辆多条 → 取“最接近且不晚于查询日”的那条
veh_col_any   = first_match_col(result.columns, VEH_ID_CANDS) or col_fzg
start_col_any = first_match_col(result.columns, REG_START_CANDS) or col_start
result = (result
          .sort_values([veh_col_any, start_col_any], ascending=[True, False])
          .drop_duplicates(subset=[veh_col_any]))

# 统一列名 & 输出
result = result.rename(columns={
    veh_col_any: "Fahrzeug_ID",
    first_match_col(result.columns, REG_REGION_CANDS) or col_reg: "Zulassungsbezirk",
    start_col_any: "Zulassungsbeginn",
})
if "Abmeldedatum" not in result.columns:
    result["Abmeldedatum"] = pd.NaT

print("\n=== Zulassungs-Ergebnisse am Unfalltag (Top 20) ===")
print(result[["Fahrzeug_ID","Zulassungsbezirk","Zulassungsbeginn","Abmeldedatum"]].head(20))

# 保存
out_dir = BASE_DIR / "outputs"
out_dir.mkdir(exist_ok=True)
out_path = out_dir / "Task6_result.csv"
result[["Fahrzeug_ID","Zulassungsbezirk","Zulassungsbeginn","Abmeldedatum","source_file"]].to_csv(
    out_path, index=False, encoding="utf-8-sig"
)
print(f"\n✅ Ergebnis gespeichert: {out_path}")


✅ Projektwurzelverzeichnis: D:\IDA\Case_Study_IDA_Group11
✅ Data-Verzeichnis: D:\IDA\Case_Study_IDA_Group11\Data
>>> Scanne Data-Verzeichnis zur Lokalisierung des Teils… (zeigt Trefferdateien, Spalten und Trefferanzahl)
    [Treffer] Fahrzeug\Bestandteile_Fahrzeuge_OEM1_Typ12.csv | Teil-Spalte=ID_Karosserie | Fahrzeug-Spalte=ID_Fahrzeug | Treffer=1
    [Treffer] Komponente\Bestandteile_Komponente_K5.csv | Teil-Spalte=ID_K5 | Fahrzeug-Spalte=None | Treffer=1
    [Treffer] Komponente\Komponente_K5.csv | Teil-Spalte=ID_Karosserie.x | Fahrzeug-Spalte=None | Treffer=1

>>> Insgesamt 3 Dateien getroffen; Anzahl gefundener Fahrzeug-IDs: 1 (Beispiel) ['12-1-12-82']

>>> Filtere gültige Zulassungen am Unfalltag (mit Fortschritt)…
    [Fortschritt] Zulassungen_alle_Fahrzeuge.csv 1,000,000 Zeilen verarbeitet…
    [Fortschritt] Zulassungen_alle_Fahrzeuge.csv 2,000,000 Zeilen verarbeitet…
    [Fortschritt] Zulassungen_alle_Fahrzeuge.csv 3,000,000 Zeilen verarbeitet…

=== Zulassungs-Ergebnisse am Un